In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.experimental import enable_halving_search_cv # explicitly require this experimental feature
from sklearn.model_selection import KFold, HalvingGridSearchCV, train_test_split, ShuffleSplit, StratifiedShuffleSplit
from sklearn.svm import LinearSVC, SVC

# Reading Features

In [212]:
# Read EEG Features
features = pd.read_csv('all_features.csv', index_col=0)
features = features.dropna(subset=['targets'])
scores_max, scores_min = max(features['targets']), min(features['targets'])
N_CLASSES = 2
for subject_session in features.index:
    score = features.loc[subject_session]['targets']
    score = int((score-1e-6 - scores_min) / (scores_max - scores_min) * N_CLASSES)  # discretise
    features.loc[subject_session, 'targets'] = score
features.loc[:, 'targets'] = features['targets'].astype(int)

# Normalize each feature column
for column in features.columns[:-1]:
    features.loc[:, column] = (features[column] - features[column].min()) / (features[column].max() - features[column].min())

In [213]:
# Remove columns completely filled with NaNs
features = features.dropna(axis=1, how='all')

In [214]:
# Make targets int
features = features.astype({'targets':'int'})

### Keep only some features?

From file? 

In [215]:
selected_features = np.loadtxt('results/optimal_hjorth_only.txt', dtype=str)

Just some groups?

In [216]:
#targets = features.loc[:, 'targets']

# Only Spectral
#features = features.iloc[:, :600]

# Only Hjorth
#features = features.iloc[:, 600:666]


###########################
#features.loc[:,'targets'] = targets  # restore targets
#features = features.dropna()  # remove NaNs

Show

In [217]:
features.head()

,C3_delta_relative_power,C3_delta_spectral_entropy,C3_delta_spectral_flatness,C3_delta_spectral_diff,C3_theta_relative_power,C3_theta_spectral_entropy,C3_theta_spectral_flatness,C3_theta_spectral_diff,C3_lowalpha_relative_power,C3_lowalpha_spectral_entropy,...,theta_T6-Fp1,theta_T6-Fp2,theta_T6-O1,theta_T6-O2,theta_T6-P3,theta_T6-P4,theta_T6-T3,theta_T6-T4,theta_T6-T5,targets
300_2,0.290847,0.506399,0.513087,0.677922,0.483301,0.961598,0.963735,0.457664,0.651047,0.916400,...,0.166667,0.268571,0.275000,0.345865,0.323308,0.255639,0.295690,0.238095,0.300752,1
042_1,0.633370,0.698376,0.703206,0.625373,0.724320,0.985624,0.986713,0.462133,0.756865,0.940794,...,0.277778,0.253333,0.183333,0.236842,0.289474,0.210526,0.041356,0.166667,0.342105,1
261_2,0.576232,0.410789,0.425846,0.240497,0.004830,0.331900,0.330336,0.516979,0.780312,0.956951,...,0.138889,0.173333,0.037500,0.000000,0.263158,0.052632,0.178305,0.166667,0.210526,0
327_1,0.364361,0.751537,0.751709,0.529935,0.640646,0.970125,0.971902,0.165480,0.790988,0.902258,...,0.141774,0.260257,0.063638,0.123380,0.107996,0.304858,0.145227,0.390704,0.466852,0
200_2,0.450184,0.595821,0.606345,0.605906,0.461242,0.985107,0.986188,0.438587,0.434333,0.742340,...,0.137315,0.247290,0.254309,0.327196,0.285001,0.183689,0.000000,0.264164,0.331669,0


# Model

In [218]:
params = {'class_weight': None,
          'criterion': 'gini',
          'max_depth': 5,
          'max_features': 'sqrt',
          'max_samples': None,
          'min_samples_leaf': 0.51,
          'min_samples_split': 5,
          'n_estimators': 500,
          'random_state': 0}
#model = RandomForestClassifier(**params)
model = RandomForestClassifier(500, max_depth=5, random_state=0) #, oob_score=True, warm_start=True)
#model = GradientBoostingClassifier(loss='exponential', n_estimators=100, criterion='friedman_mse', max_depth=3, learning_rate=0.1, random_state=0)
#model = SVC(kernel='rbf', C=5, class_weight='balanced', random_state=0)

# Run Purpose

Cross-Validation

In [219]:
CV = True
# 10-fold cross-validation
#cv = StratifiedShuffleSplit(n_splits=10, random_state=0)
#cv = StratifiedKFold(n_splits=10)  # sem shuffle
#cv = ShuffleSplit(n_splits=10, random_state=0)  # sem estratificação
#cv = KFold(n_splits=5)  # sem shuffle nem estratificação
# Leave-4-out cross-validation
#cv = StratifiedShuffleSplit(n_splits=179-4, random_state=0)
#cv = ShuffleSplit(n_splits=179-4, random_state=0)  # sem estratificação
#cv = KFold(n_splits=len(features)-4)  # sem shuffle nem estratificação
cv = KFold(n_splits=len(features) // 4)  # sem shuffle nem estratificação
# My cross-validation
#cv = StratifiedLeavePOut(C=2, P=4, with_repetition=False)

In [220]:
# Sort subjects?
features = features.sort_index()
# Remove subjects without session pair?
features = features.drop(labels=['005_2', '102_2', '109_1', '195_2', '303_2'])
features.dropna(axis=0, inplace=True)  # Discard subjects with NaNs
features

,C3_delta_relative_power,C3_delta_spectral_entropy,C3_delta_spectral_flatness,C3_delta_spectral_diff,C3_theta_relative_power,C3_theta_spectral_entropy,C3_theta_spectral_flatness,C3_theta_spectral_diff,C3_lowalpha_relative_power,C3_lowalpha_spectral_entropy,...,theta_T6-Fp1,theta_T6-Fp2,theta_T6-O1,theta_T6-O2,theta_T6-P3,theta_T6-P4,theta_T6-T3,theta_T6-T4,theta_T6-T5,targets
003_1,0.442878,0.596282,0.595396,0.296637,0.504101,0.980879,0.982634,0.492826,0.730751,0.853720,...,1.333334e-01,0.360000,0.125000,0.178947,0.336842,0.178947,0.506983,0.266667,0.336842,0
003_2,0.486413,0.804029,0.805181,0.725483,0.751756,0.973147,0.975185,0.337463,0.700120,0.882486,...,3.095238e-01,0.222857,0.100000,0.233083,0.142857,0.233083,0.248736,0.166667,0.165414,0
004_1,0.552516,0.871960,0.873773,0.728517,0.211744,0.709394,0.711538,0.551936,0.553620,0.914040,...,4.000000e-01,0.456000,0.335000,0.400000,0.557895,0.178947,0.309776,0.300000,0.526316,0
004_2,0.371168,0.438033,0.441447,0.210781,0.271195,0.803898,0.809099,0.345361,0.631242,0.924679,...,5.555555e-01,0.733333,0.475000,0.578947,0.842105,0.421053,0.616542,0.777778,0.736842,0
008_1,0.279384,0.120382,0.126638,0.040625,0.386846,0.983939,0.984982,0.610186,0.150069,0.857516,...,4.444444e-01,0.306667,0.358333,0.421053,0.526316,0.315790,0.397424,0.444444,0.578947,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351_2,0.599715,0.706324,0.709676,0.625200,0.859793,0.988145,0.988805,0.357386,0.702661,0.626659,...,6.645887e-09,0.066667,0.066667,0.289474,0.210526,0.210526,0.178305,0.027778,0.289474,0
355_1,0.469039,0.597319,0.598046,0.268693,0.469655,0.969750,0.971496,0.715936,0.079379,0.630700,...,9.523808e-02,0.314286,0.200000,0.210526,0.120301,0.255639,0.319167,0.142857,0.165414,1
355_2,0.636086,0.841833,0.844974,0.537610,0.732354,0.944329,0.947996,0.902838,0.057219,0.663621,...,3.666667e-01,0.296000,0.335000,0.305263,0.336842,0.431579,0.276909,0.266667,0.494737,1
356_1,0.655786,0.853059,0.853764,0.642454,0.678495,0.941064,0.943437,0.322846,0.934990,0.892372,...,3.095238e-01,0.222857,0.250000,0.300752,0.255639,0.233083,0.060920,0.357143,0.413534,0


Selection

In [221]:
SELECTION = True
selector = RFECV
min_features_to_select = 10
max_features_to_select = 24
n_features_to_select = 24

Hyperparameters Tuning

In [222]:
HYPERPARAMETERS_TUNING = False
tunner = HalvingGridSearchCV
hyperparameters = (
    {'n_estimators': [100, 500, 1000],
     'criterion': ['gini', 'entropy', 'log_loss'],
     'max_depth': [5, 10, 20, None],
     'min_samples_split': [5, 10, 0.51],
     'min_samples_leaf': [1, 2, 0.51],
     'max_features': ['sqrt', 'log2'],
     'class_weight': [None, 'balanced', 'balanced_subsample'],
     'max_samples': [None, 0.5, 0.75, 0.9],
     }
)  # n_candidates: 7776

Sequential Selection

In [223]:
SEQUENTIAL_SELECTION = False
additional_features = None  # pli_features

# Evaluation

In [224]:
if SELECTION and CV and not HYPERPARAMETERS_TUNING:
    selector_cv = selector(estimator=model, step=1, cv=cv,
                            scoring='f1_weighted', min_features_to_select=min_features_to_select,
                            verbose=2, n_jobs=-1)

    selector_cv.fit(features.iloc[:, :-1], features['targets'])
    print(f"Optimal number of features: {selector_cv.n_features_}")

    # Plot number of features VS. cross-validation scores
    n_scores = len(selector_cv.cv_results_["mean_test_score"])
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Mean test accuracy")
    plt.scatter(
        range(min_features_to_select, n_scores + min_features_to_select),
        selector_cv.cv_results_["mean_test_score"],
        #yerr=selector_cv.cv_results_["std_test_score"],
    )
    plt.title("Recursive Feature Elimination \nwith correlated features")
    plt.show()

    # Get optimal features
    selected_features = selector_cv.get_feature_names_out()

ValueError: when `importance_getter=='auto'`, the underlying estimator SVC should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.

In [ ]:
if SELECTION and not CV and not HYPERPARAMETERS_TUNING:
    #selector = RFE(estimator=model, n_features_to_select=100, step=1, verbose=2)
    #selector = selector(estimator=model, max_features=max_features_to_select)
    selector = selector(LinearSVC(dual=True, penalty="l2", loss='hinge'), max_features=max_features_to_select)
    selector.fit(features.iloc[:, :-1], features['targets'])
    selected_features = selector.get_feature_names_out()

In [ ]:
selected_features

In [ ]:
features = features[np.append(selected_features, ['targets', ])]  # keep only the selected features and the targets

In [ ]:
features.dropna(axis=0, inplace=True)  # Discard subjects with NaNs

In [ ]:
features

In [ ]:
if CV and not SELECTION and not HYPERPARAMETERS_TUNING:
    from sklearn.model_selection import cross_val_score
    #print("Cross-Validation with StratifiedLeavePOut")
    #print("Number of splits:", cv.get_n_splits(features['targets']))
    #print("Size of test sets", cv.p * cv.c * 2)
    scores = cross_val_score(model, features[selected_features], features['targets'],
                              cv=cv, scoring='f1_weighted', verbose=2, n_jobs=-1)
    print("Cross-Validation mean score:", scores.mean())
    print("Cross-Validation std score:", scores.std())
    print("Cross-Validation max score:", scores.max())
    print("Cross-Validation min score:", scores.min())

In [ ]:
if not CV and not SELECTION and not HYPERPARAMETERS_TUNING and not SEQUENTIAL_SELECTION:
    # Split subjects into train and test (using sklearn)
    train_size = 0.98
    n_train = int(len(features) * train_size)
    n_test = len(features) - n_train
    train_dataset, test_dataset = train_test_split(features, train_size=train_size,
                                                   shuffle=True,
                                                   #stratify=features['targets'],
                                                   random_state=1)
    train_features = train_dataset[selected_features]
    train_targets = train_dataset['targets']
    test_features = test_dataset[selected_features]
    test_targets = test_dataset['targets']
    print("Train features shape:", train_features.shape)
    print("Test features shape:", test_features.shape)

    # Train
    model = model.fit(train_features, train_targets)

    #test_features = train_features  # FIXME: remove this line
    #test_targets = train_targets  # FIXME: remove this line

    # Test
    predictions = model.predict(test_features)
    # Adjust predictions to the number of classes
    #predictions_min, predictions_max = 0, 1
    #for i in range(len(predictions)):
    #    predictions[i] = int((predictions[i]-1e-6 - predictions_min) / (predictions_max - predictions_min) * N_CLASSES)  # discretise

    # 7.1) Accuracy
    accuracy = accuracy_score(test_targets, predictions)

    # 7.2) F1-Score
    f1 = f1_score(test_targets, predictions, average='weighted')

    print(f'Accuracy: {accuracy}')
    print(f'F1-Score: {f1}')
    print('-----\n')

    # 8. Plot Confusion Matrix
    from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
    import seaborn as sn
    import pandas as pd
    import matplotlib.pyplot as plt

    cm = confusion_matrix(test_targets, predictions)
    df_cm = pd.DataFrame(cm, range(N_CLASSES), range(N_CLASSES))
    plt.figure(figsize=(10, 7))
    sn.set(font_scale=1.4)  # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})  # font size
    plt.show()

In [ ]:
if HYPERPARAMETERS_TUNING and CV and not SELECTION and not SEQUENTIAL_SELECTION:
    print("Hyperparameter tuning")
    #_tunner = tunner(model, hyperparameters, resource='n_samples', min_resources=int(179*0.5),
    #                 cv=cv, scoring='f1_weighted', error_score=0, random_state=0,
    #                 verbose=2, n_jobs=-1)

    _tunner = tunner(model, hyperparameters,
                     cv=cv, scoring='f1_weighted', error_score=0, random_state=0,
                     verbose=2, n_jobs=-1)

    _tunner.fit(features[selected_features], features['targets'])
    print("Finished hyperparameter tuning")
    print("Best parameters:", _tunner.best_params_)
    print("Best score:", _tunner.best_score_)
    print("All results:")
    all_results = pd.DataFrame(_tunner.cv_results_)
    print(all_results)

In [ ]:
if SEQUENTIAL_SELECTION and not SELECTION and not CV and not HYPERPARAMETERS_TUNING:

    def _train_test(model, experimental_features, train_size=0.85):
        train_dataset, test_dataset = train_test_split(features, train_size=train_size, shuffle=True, stratify=features['targets'], random_state=0)
        train_features, train_targets = train_dataset[experimental_features], train_dataset['targets']
        test_features, test_targets = test_dataset[experimental_features], test_dataset['targets']
        model.fit(train_features, train_targets)
        predictions = model.predict(test_features)
        f1 = f1_score(test_targets, predictions, average='weighted')
        return f1

    # Baseline test
    f1_baseline = _train_test(RandomForestClassifier(**params), selected_features)

    # Iterate through 'additional_features' and add the one that improves the most the model
    print("Sequential feature selection")
    results = {'baseline': f1_baseline}
    for feature in additional_features.columns:
        experimental_set_features = np.append(selected_features, feature)
        f1 = _train_test(RandomForestClassifier(**params), experimental_set_features, train_size=0.98)
        print(f"With '{feature}'\nF1-score = {f1}\n")
        results[feature] = f1